In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
path = '../data/' 
hotels = pd.read_csv(path + '/features_hotels.csv')
data = pd.read_csv(path + 'data.csv')
data_test = pd.read_csv(path + 'test_set.csv')

In [3]:
to_exclude = data[['city', 'language','mobile','request_number','date']].drop_duplicates()
# to_exclude.to_csv('to_exclude.csv', index = False)
print('Nombre de requêtes uniques :', to_exclude.shape[0])

Nombre de requêtes uniques : 8329


In [4]:
tot = 0 
for i in range(7):
    tot += np.unique(data.loc[data.request_number == i].avatar_id.values).shape[0]

print('Estimation du nombre total de requêtes :', tot)

Estimation du nombre total de requêtes : 11171


In [5]:
# exclusion des avatar posant problème
# data = data.drop(index = data.loc[data.avatar_id == 134].index)
# to_drop = [108,134,135,136,137,138,141]
to_drop = [134]
data = data.loc[~data.avatar_id.isin(to_drop)]

In [6]:
data = data.drop(['avatar_id'], axis = 1).drop_duplicates()

In [7]:
# ajout des caractéristiques des hotels
data = data.merge(hotels, on=['hotel_id','city'])
data_test = data_test.merge(hotels, on=['hotel_id','city'])
data_test = data_test.sort_values('index').reset_index(drop=True).drop(['index'], axis = 1)

In [8]:
# création de la colonne request_number dans le test set
data_test['request_number'] = 1
for avatar in np.unique(data_test['avatar_id']):
    data_test.loc[data_test['avatar_id'] == avatar, 'request_number'] = data_test['order_requests'].loc[data_test['avatar_id']== avatar] - min(data_test['order_requests'].loc[data_test['avatar_id']== avatar])+1

In [9]:
# ajout de la variable ville == langue
lang_cit = True

if lang_cit == True: 
    dic_lang = {'amsterdam':'dutch', 'copenhagen':'danish', 'madrid':'spanish', 'paris':'french', 'rome':'italian', 'sofia':'bulgarian', 'valletta':'maltese', 'vienna':'austrian' ,'vilnius':'lithuanian'}
    data['city_language'] = data['city'].map(dic_lang)
    data['is_same_cl'] = data['city_language']==data['language']
    data_test['city_language'] = data_test['city'].map(dic_lang)
    data_test['is_same_cl'] = data_test['city_language']==data_test['language']

In [10]:
colQT = ['date', 'request_number', 'stock','price']
colQL = ['city', 'language', 'mobile', 'group', 'brand', 'parking', 'pool', 'children_policy', 'is_same_cl']

In [11]:
encoder = OneHotEncoder()

train_data = encoder.fit_transform(data[colQL]).toarray()

test_data = encoder.transform(data_test[colQL]).toarray()

X_train = np.concatenate((train_data, data[colQT[:-1]].values), axis = 1)
X_test = np.concatenate((test_data, data_test[colQT[:-1]].values), axis = 1)

y = data.price.values

In [12]:
X_no_anomaly = X_test.copy()
X_with_anomalies = X_train.copy()

In [ ]:
from sklearn.cluster import MeanShift

# Entraîner Mean-Shift sur les données labellisées valides
meanshift = MeanShift(n_jobs=-1)
meanshift.fit(X_with_anomalies)

In [ ]:
np.unique(meanshift.labels_)

In [ ]:
# Prédire les clusters pour les données dans lesquelles vous souhaitez détecter les anomalies
predictions = meanshift.predict(X_no)

In [ ]:
np.unique(predictions)

In [ ]:
# Identifier les observations qui sont considérées comme des anomalies (i.e., qui sont classées dans un cluster différent de ceux formés à partir des données labellisées valides)
anomalies = [i for i, y in enumerate(predictions) if y not in meanshift.labels_]

In [ ]:
anomalies